In [2]:
import os
import logging
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases, LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

class TopicModeler:
    def __init__(self, input_path):
        self.input_path = input_path
        self.docs = []
        self.filtered_docs = []
        self.dictionary = None
        self.corpus = None
        self.model = None
        self.coherence_scores = []
        self.perplexity_scores = []
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.lemmatizer = WordNetLemmatizer()
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    def load_documents(self):
        for filename in os.listdir(self.input_path):
            file_path = os.path.join(self.input_path, filename)
            if os.path.isfile(file_path) and filename.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if isinstance(content, str):
                        self.docs.append(content)
        print(f"Loaded {len(self.docs)} documents")
        if self.docs:
            print(f"Sample document: {self.docs[0][:100]}...")

    def preprocess_documents(self):
        self.docs = [[token.lower() for token in self.tokenizer.tokenize(doc)] for doc in self.docs if isinstance(doc, str)]
        self.docs = [[token for token in doc if not token.isnumeric() and len(token) > 1] for doc in self.docs]
        stop_words = set(stopwords.words('english'))
        self.docs = [[word for word in doc if word not in stop_words] for doc in self.docs]
        self.docs = [[self.lemmatizer.lemmatize(token) for token in doc] for doc in self.docs]
        print(f"After preprocessing: {len(self.docs)} documents")
        if self.docs:
            print(f"Sample preprocessed document: {self.docs[0][:10]}...")

    def add_bigrams(self):
        bigram = Phrases(self.docs, min_count=5, threshold=100)
        for idx in range(len(self.docs)):
            for token in bigram[self.docs[idx]]:
                if '_' in token:
                    self.docs[idx].append(token)
        print(f"Added bigrams. Sample document: {self.docs[0][:15]}...")

    def filter_with_tfidf(self, top_n=200, bottom_n=50):
        texts = [' '.join(doc) for doc in self.docs]
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        mean_tfidf_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
        
        top_indices = mean_tfidf_scores.argsort()[-top_n:]
        bottom_indices = mean_tfidf_scores.argsort()[:bottom_n]
        
        words_to_remove = set(feature_names[i] for i in np.concatenate([top_indices, bottom_indices]))

        self.filtered_docs = [[word for word in doc if word not in words_to_remove] for doc in self.docs]
        print(f"Filtered documents using TF-IDF. Sample filtered document: {self.filtered_docs[0][:10]}...")

    def create_dictionary_and_corpus(self):
        self.dictionary = Dictionary(self.filtered_docs)
        original_size = len(self.dictionary)
        self.dictionary.filter_extremes(no_below=2, no_above=0.9)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.filtered_docs]
        print(f"Dictionary size: {len(self.dictionary)} (reduced from {original_size})")
        print(f"Corpus size: {len(self.corpus)}")
        if self.corpus:
            print(f"Sample corpus entry: {self.corpus[0][:10]}")

    def train_lda_model(self, num_topics=6, chunksize=2000, passes=20, iterations=400, alpha='auto', eta='auto'):
        if not self.corpus or not self.dictionary:
            raise ValueError("Corpus or dictionary is empty. Check your preprocessing steps.")
        
        print(f"Training LDA model with num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}, alpha={alpha}, eta={eta}")
        
        self.model = LdaModel(
            corpus=self.corpus,
            id2word=self.dictionary,
            chunksize=chunksize,
            alpha=alpha,
            eta=eta,
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=None,
            random_state=42
        )
        print("LDA model trained successfully")

    def print_model_info(self):
        print('Number of unique tokens: %d' % len(self.dictionary))
        print('Number of documents: %d' % len(self.corpus))
        top_topics = self.model.top_topics(self.corpus)
        avg_topic_coherence = sum([t[1] for t in top_topics]) / self.model.num_topics
        print('Average topic coherence: %.4f.' % avg_topic_coherence)
        print("\nTop topics:")
        pprint(self.model.print_topics())
        print('\nPerplexity: ', self.model.log_perplexity(self.corpus))
        perplexity=self.model.log_perplexity(self.corpus)
        coherence_model_lda = CoherenceModel(model=self.model, texts=self.filtered_docs, dictionary=self.dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)
        return coherence_lda,perplexity

    def visualize_topics(self):
        vis = pyLDAvis.gensim.prepare(self.model, self.corpus, self.dictionary)
        return vis

    def analyze_documents(self):
        for i, doc in enumerate(self.filtered_docs[:5]):
            bow = self.dictionary.doc2bow(doc)
            doc_topics = self.model.get_document_topics(bow)
            print(f"\nDocument {i} topics:")
            pprint(doc_topics)
            best_topic = max(doc_topics, key=lambda x: x[1])
            print(f"Best topic: {best_topic[0]}")
            print(f"Top words in this topic:")
            pprint(self.model.show_topic(best_topic[0]))
            print(f"Original document: {' '.join(doc[:30])}...")
            print()

    def save_model(self, file_path):
        if self.model is None:
            raise ValueError("No model has been trained yet.")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        self.model.save(file_path)
        print(f"Model saved to {file_path}")

    def load_model(self, file_path):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"No model file found at {file_path}")
        self.model = LdaModel.load(file_path)
        print(f"Model loaded from {file_path}")

    def save_dictionary_and_corpus(self, dict_path, corpus_path):
        if self.dictionary is None or self.corpus is None:
            raise ValueError("Dictionary and corpus have not been created yet.")
        os.makedirs(os.path.dirname(dict_path), exist_ok=True)
        os.makedirs(os.path.dirname(corpus_path), exist_ok=True)
        self.dictionary.save(dict_path)
        with open(corpus_path, 'wb') as f:
            pickle.dump(self.corpus, f)
        print(f"Dictionary saved to {dict_path}")
        print(f"Corpus saved to {corpus_path}")

    def load_dictionary_and_corpus(self, dict_path, corpus_path):
        if not os.path.exists(dict_path) or not os.path.exists(corpus_path):
            raise FileNotFoundError(f"Dictionary or corpus file not found.")
        self.dictionary = Dictionary.load(dict_path)
        with open(corpus_path, 'rb') as f:
            self.corpus = pickle.load(f)
        print(f"Dictionary loaded from {dict_path}")
        print(f"Corpus loaded from {corpus_path}")

    def plot_scores(self, iteration):
        num_topics = [x[0] for x in self.coherence_scores]
        coherences = [x[1] for x in self.coherence_scores]
        perplexities = [x[1] for x in self.perplexity_scores]  # Extract perplexities

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(num_topics, coherences, marker='o', label='Coherence')
        plt.xlabel('Number of Topics')
        plt.ylabel('Coherence Score')
        plt.title('Coherence Score by Number of Topics')
        plt.grid(True)
        plt.xticks(num_topics)
        plt.ylim(0, max(coherences) * 1.1)
        
        plt.subplot(1, 2, 2)
        plt.plot(num_topics, perplexities, marker='o', color='r', label='Perplexity')
        plt.xlabel('Number of Topics')
        plt.ylabel('Perplexity')
        plt.title('Perplexity by Number of Topics')
        plt.grid(True)
        plt.xticks(num_topics)

        plt.tight_layout()
        plt.savefig(f'coherence_perplexity_scores_iteration_{iteration}.png')
        plt.close()

    def get_word_frequencies(self):
        topics = self.model.get_topics()
        word_frequencies = np.sum(topics, axis=0)
        word_frequencies /= self.model.num_topics
        return {self.dictionary[i]: freq for i, freq in enumerate(word_frequencies)}

    def get_low_frequency_words(self, threshold=0.01):
        word_frequencies = self.get_word_frequencies()
        return {word: freq for word, freq in word_frequencies.items() if freq < threshold}

    def get_high_frequency_words(self, threshold=0.00016):
        word_frequencies = self.get_word_frequencies()
        high_freq_words = {word: freq for word, freq in word_frequencies.items() if freq > threshold}
        
        if not high_freq_words:
            print(f"No words found with a frequency higher than the threshold of {threshold}.")
        
        return high_freq_words


    def save_frequency_words(self, low_freq_file, high_freq_file, low_threshold=0.01, high_threshold=0.1):
        low_freq_words = self.get_low_frequency_words(low_threshold)
        high_freq_words = self.get_high_frequency_words(high_threshold)

        df_low = pd.DataFrame(list(low_freq_words.items()), columns=['Word', 'Frequency'])
        df_low = df_low.sort_values('Frequency', ascending=True)
        df_low.to_csv(low_freq_file, index=False)
        print(f"Low frequency words saved to {low_freq_file}")

        df_high = pd.DataFrame(list(high_freq_words.items()), columns=['Word', 'Frequency'])
        df_high = df_high.sort_values('Frequency', ascending=False)
        df_high.to_csv(high_freq_file, index=False)
        print(f"High frequency words saved to {high_freq_file}")

    def run_with_params(self, num_topics, chunksize, passes, iterations):
        try:
            self.load_documents()
            self.preprocess_documents()
            self.add_bigrams()
            self.filter_with_tfidf(top_n=200, bottom_n=50)
            self.create_dictionary_and_corpus()
            self.train_lda_model(num_topics=num_topics, chunksize=chunksize, passes=passes, iterations=iterations)
            model_dir = f"lda_models/topics_{num_topics}_passes_{passes}_iterations_{iterations}"
            self.save_model(os.path.join(model_dir, 'trained_model'))
            self.save_dictionary_and_corpus(os.path.join(model_dir, 'dictionary'), os.path.join(model_dir, 'corpus'))
            coherence_lda, perplexity = self.print_model_info()  # Capture perplexity
            self.coherence_scores.append((num_topics, coherence_lda))
            self.perplexity_scores.append((num_topics, perplexity))  # Store perplexity
            
            # Plot and save coherence and perplexity score after each run
            self.plot_scores(len(self.coherence_scores))
            
            vis = self.visualize_topics()
            pyLDAvis.save_html(vis, os.path.join(model_dir, 'lda_visualization.html'))
            self.analyze_documents()
            
            return vis
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            raise

if __name__ == "__main__":
    modeler = TopicModeler('txt')

    param_combinations = [
        (15, 2000, 10, 200),
        (30, 2000, 10, 400),
        (45, 2000, 20, 200),
        (60, 2000, 20, 400),
        (75, 2000, 20, 400),
        (90, 2000, 20, 400)
    ]

    csv_folder = 'csv_output'
    os.makedirs(csv_folder, exist_ok=True)

    for num_topics, chunksize, passes, iterations in param_combinations:
        print(f"\nRunning model with parameters: num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}")
        modeler.run_with_params(num_topics, chunksize, passes, iterations)
    
    modeler.save_frequency_words(
        os.path.join(csv_folder, 'low_freq_words.csv'),
        os.path.join(csv_folder, 'high_freq_words.csv'),
        low_threshold=0.01,
    high_threshold=0.00016
    )


Running model with parameters: num_topics=15, chunksize=2000, passes=10, iterations=200
Loaded 440 documents
Sample document: Pricing for interconnection related to the provision of number portability, as referred to in Articl...


2024-08-06 18:19:24,839 : INFO : collecting all words and their counts
2024-08-06 18:19:24,840 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-08-06 18:19:24,862 : INFO : collected 23445 token types (unigram + bigrams) from a corpus of 44718 words and 440 sentences
2024-08-06 18:19:24,862 : INFO : merged Phrases<23445 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-08-06 18:19:24,863 : INFO : Phrases lifecycle event {'msg': 'built Phrases<23445 vocab, min_count=5, threshold=100, max_vocab_size=40000000> in 0.02s', 'datetime': '2024-08-06T18:19:24.863347', 'gensim': '4.3.2', 'python': '3.11.9 (main, Jul 16 2024, 11:07:54) [Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5-arm64-arm-64bit', 'event': 'created'}
2024-08-06 18:19:24,938 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-08-06 18:19:24,948 : INFO : built Dictionary<3416 unique tokens: ['abstract', 'adoption', 'advance', 'appeal', 'body']...> from 440 do

After preprocessing: 440 documents
Sample preprocessed document: ['pricing', 'interconnection', 'related', 'provision', 'number', 'portability', 'referred', 'article', 'directive', 'ec']...
Added bigrams. Sample document: ['pricing', 'interconnection', 'related', 'provision', 'number', 'portability', 'referred', 'article', 'directive', 'ec', 'european', 'parliament', 'council', 'march', 'universal']...
Filtered documents using TF-IDF. Sample filtered document: ['pricing', 'interconnection', 'related', 'number', 'portability', 'concern', 'traffic', 'number', 'ported', 'set']...
Dictionary size: 1835 (reduced from 3416)
Corpus size: 440
Sample corpus entry: [(0, 1), (1, 1), (2, 3), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
Training LDA model with num_topics=15, chunksize=2000, passes=10, iterations=200, alpha=auto, eta=auto


2024-08-06 18:19:25,119 : INFO : optimized alpha [0.05601553, 0.055268753, 0.0526802, 0.054346085, 0.054080088, 0.053848427, 0.053853672, 0.051989175, 0.05495826, 0.056449115, 0.05256947, 0.0520408, 0.05585058, 0.054048866, 0.05386136]
2024-08-06 18:19:25,121 : INFO : topic #7 (0.052): 0.012*"device" + 0.009*"charge" + 0.008*"equipment" + 0.008*"licence" + 0.007*"paid" + 0.006*"authorisation" + 0.006*"vehicle" + 0.006*"damage" + 0.006*"year" + 0.005*"however"
2024-08-06 18:19:25,121 : INFO : topic #11 (0.052): 0.012*"operator" + 0.010*"damage" + 0.010*"wine" + 0.008*"notification" + 0.007*"established" + 0.007*"accommodation" + 0.007*"description" + 0.007*"used" + 0.006*"property" + 0.006*"third"
2024-08-06 18:19:25,122 : INFO : topic #12 (0.056): 0.008*"read_conjunction" + 0.008*"conjunction" + 0.007*"set" + 0.007*"take" + 0.007*"required" + 0.006*"water" + 0.006*"conformity" + 0.005*"place" + 0.005*"packaging" + 0.005*"second"
2024-08-06 18:19:25,122 : INFO : topic #0 (0.056): 0.009*

LDA model trained successfully
Model saved to lda_models/topics_15_passes_10_iterations_200/trained_model
Dictionary saved to lda_models/topics_15_passes_10_iterations_200/dictionary
Corpus saved to lda_models/topics_15_passes_10_iterations_200/corpus
Number of unique tokens: 1835
Number of documents: 440
Average topic coherence: -10.8608.

Top topics:
[(0,
  '0.014*"call" + 0.012*"emergency" + 0.011*"number" + 0.008*"single" + '
  '0.007*"telephone" + 0.007*"caller" + 0.007*"lender" + 0.007*"available" + '
  '0.007*"make" + 0.006*"provide"'),
 (1,
  '0.010*"protected_designation" + 0.010*"registration" + 0.010*"agricultural" '
  '+ 0.009*"designation_origin" + 0.009*"grand" + 0.009*"grand_duchy" + '
  '0.009*"luxembourg" + 0.009*"duchy" + 0.008*"trade_mark" + 0.008*"fact"'),
 (2,
  '0.009*"framework" + 0.009*"ensure" + 0.008*"procedure" + 0.008*"regulatory" '
  '+ 0.008*"january" + 0.007*"registration" + 0.007*"domestic" + '
  '0.007*"defect" + 0.007*"drink" + 0.007*"spirit"'),
 (3,
 

...

LDA model trained successfully
Model saved to lda_models/topics_90_passes_20_iterations_400/trained_model
Dictionary saved to lda_models/topics_90_passes_20_iterations_400/dictionary
Corpus saved to lda_models/topics_90_passes_20_iterations_400/corpus
Number of unique tokens: 1835
Number of documents: 440
Average topic coherence: -8.1622.

Top topics:
[(84,
  '0.001*"judicial" + 0.001*"body" + 0.001*"following" + 0.001*"previously" + '
  '0.001*"serious" + 0.001*"includes" + 0.001*"place" + 0.001*"position" + '
  '0.001*"subsequent" + 0.001*"selection"'),
 (86,
  '0.048*"element" + 0.039*"finding" + 0.027*"annulment" + 0.024*"consequence" '
  '+ 0.020*"amendment" + 0.020*"restoration" + 0.020*"objective" + '
  '0.020*"removing" + 0.020*"lead" + 0.020*"altering"'),
 (48,
  '0.042*"ceiling" + 0.042*"instrument" + 0.021*"exceed" + 0.021*"set" + '
  '0.021*"signed" + 0.021*"civil" + 0.021*"convention" + 0.021*"behalf" + '
  '0.021*"transposing" + 0.021*"determined"'),
 (11,
  '0.068*"exten

2024-08-06 18:20:17,965 : INFO : -7.614 per-word bound, 195.8 perplexity estimate based on a held-out corpus of 440 documents with 16915 words



Perplexity:  -7.613544123660979


2024-08-06 18:20:18,378 : INFO : -7.614 per-word bound, 195.8 perplexity estimate based on a held-out corpus of 440 documents with 16915 words
2024-08-06 18:20:18,422 : INFO : using ParallelWordOccurrenceAccumulator<processes=9, batch_size=64> to estimate probabilities from sliding windows
2024-08-06 18:20:20,828 : INFO : 9 accumulators retrieved from output queue
2024-08-06 18:20:21,116 : INFO : accumulated word occurrence stats for 1774 virtual documents



Coherence Score:  0.4538564207186806

Document 0 topics:
[(69, 0.9868045)]
Best topic: 69
Top words in this topic:
[('appeal', 0.039537888),
 ('operator', 0.02376595),
 ('number', 0.02376595),
 ('examine', 0.023765948),
 ('hearing', 0.023765946),
 ('dispute', 0.015873138),
 ('set', 0.015873138),
 ('portability', 0.015873138),
 ('body', 0.015873138),
 ('regulatory', 0.015873138)]
Original document: pricing interconnection related number portability concern traffic number ported set incurred mobile telephone operator implement request number porting adoption laying method used calculating fix advance abstract model maximum charged donor...


Document 1 topics:
[(87, 0.9696061)]
Best topic: 87
Top words in this topic:
[('united', 0.04347172),
 ('united_kingdom', 0.043471716),
 ('milk', 0.037284274),
 ('di', 0.037265968),
 ('natural', 0.031074615),
 ('natural_person', 0.031074615),
 ('pdo', 0.0248791),
 ('northern', 0.024871359),
 ('ireland', 0.02365773),
 ('system', 0.021143978)]
Origina